In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import tensorflow as tf
from surprise.model_selection import train_test_split

In [2]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [4]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [5]:
answers = {}

In [6]:
# Some data structures that will be useful

In [7]:
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

In [8]:
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

In [9]:
##################################################
# Play prediction                                #
##################################################

In [10]:
hoursTrain[0]

('u70666506',
 'g49368897',
 {'userID': 'u70666506',
  'early_access': False,
  'hours': 63.5,
  'hours_transformed': 6.011227255423254,
  'found_funny': 1,
  'text': 'If you want to sit in queue for 10-20min and have 140 ping then this game is perfect for you :)',
  'gameID': 'g49368897',
  'user_id': '76561198030408772',
  'date': '2017-05-20'})

In [11]:
# Any other preprocessing...
users = set() # maintain a set of unique users
games = set() # maintain a set of unique games
pos_pairs = []
for l in readGz("train.json.gz"):
    u,g = l['userID'], l['gameID']
    users.add(u)
    games.add(g)
    pos_pairs += [(u, g)]

In [12]:
train_data = pos_pairs[:165000]
valid_data = pos_pairs[165000:] # because pos_pairs has length 175000

In [13]:
len(users), len(games)

(6710, 2437)

In [14]:
# Randomly sample games that weren't played
neg_pairs = set()
len_valid = len(hoursValid)
while True:
    user = random.sample(users, 1)[0]
    game = random.sample(games, 1)[0]
    if (user, game) not in pos_pairs:
        neg_pairs.add((user, game))
    if len(neg_pairs) == len_valid:
        break

/var/folders/8f/_6h5xprs32dfmm3jfs0g7k880000gn/T/ipykernel_25869/759351938.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  user = random.sample(users, 1)[0]
/var/folders/8f/_6h5xprs32dfmm3jfs0g7k880000gn/T/ipykernel_25869/759351938.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  game = random.sample(games, 1)[0]


In [15]:
len(neg_pairs), len(train_data)

(9999, 165000)

In [16]:
### Question 1

In [17]:
# Evaluate baseline strategy
gameCount = defaultdict(int)
totalPlayed = 0

for user,game in train_data:
    gameCount[game] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPlayed/2: break

predictions = []
for user,game in valid_data:
    if game in return1:
        predictions += [1]
    else:
        predictions += [0]
    
for user,game in neg_pairs:
    if game in return1:
        predictions += [1]
    else:
        predictions += [0]

sum_pred = sum([predictions[i]==1 for i in range(9999)]) + \
    sum([predictions[i] == 0 for i in range(9999, len(predictions))])
acc = sum_pred / (2*len(valid_data))
acc

0.68105

In [18]:
answers['Q1'] = acc
answers['Q1']

0.68105

In [19]:
assertFloat(answers['Q1'])

In [20]:
### Question 2

In [21]:
# Improved strategy

In [22]:
# Evaluate baseline strategy
gameCount = defaultdict(int)
totalPlayed = 0

for user,game in train_data:
    gameCount[game] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPlayed/1.5: break # Choose 67% percentile instead

predictions = []
for user,game in valid_data:
    if game in return1:
        predictions += [1]
    else:
        predictions += [0]
    
for user,game in neg_pairs:
    if game in return1:
        predictions += [1]
    else:
        predictions += [0]

sum_pred_q2 = sum([predictions[i]==1 for i in range(9999)]) + \
    sum([predictions[i] == 0 for i in range(9999, len(predictions))])
acc_q2 = sum_pred_q2 / (2*len(valid_data))
acc_q2

0.7002

In [23]:
answers['Q2'] = [1/1.5, acc_q2]
answers['Q2']

[0.6666666666666666, 0.7002]

In [24]:
assertFloatList(answers['Q2'], 2)

In [25]:
### Question 3/4

In [26]:
userPerGame = defaultdict(set) # Maps a game to the users who played it
gamePerUser = defaultdict(set) # Maps a user to the game that they played
hoursDict = {} # To retrieve an hour for a specific user/game pair

for d in hoursTrain:
    user,game = d[0], d[1]
    userPerGame[game].add(user)
    gamePerUser[user].add(game)
    hoursDict[(user, game)] = d[2]['hours']

In [27]:
userAverages = {}
gameAverages = {}

for u in gamePerUser:
    rs = [hoursDict[(u,g)] for g in gamePerUser[u]]
    userAverages[u] = sum(rs) / len(rs)

for g in userPerGame:
    rs = [hoursDict[(u,g)] for u in userPerGame[g]]
    gameAverages[g] = sum(rs) / (len(rs))

In [28]:
reviewsPerUser = defaultdict(list)
reviewsPerGame = defaultdict(list)

for d in hoursTrain:
    user,game = d[0], d[1]
    reviewsPerUser[user].append(d)
    reviewsPerGame[game].append(d)

In [29]:
reviewsPerUser["u05450000"][0]

('u05450000',
 'g52077802',
 {'hours': 16.8,
  'gameID': 'g52077802',
  'hours_transformed': 4.153805336079036,
  'early_access': False,
  'date': '2011-05-28',
  'text': 'A poorly optimized and poorly realized piece of♥♥♥♥♥♥ Stay clear of this and all future Splash Damage products.',
  'userID': 'u05450000'})

In [30]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

def predictLabel(user, game, threshold):
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d[1]
        if i2 == game: continue
        similarities.append(Jaccard(userPerGame[game], userPerGame[i2]))
    return 1 if max(similarities) > threshold else 0

In [31]:
# Evaluate baseline strategy
predictions = []
for user,game in valid_data:
    predictions.append(predictLabel(user, game, 0.03))
# print(len(predictions))
for user,game in neg_pairs:
    predictions.append(predictLabel(user, game, 0.03))
# print(len(predictions))

sum_pred_q3 = sum([predictions[i]==1 for i in range(10000)]) + \
    sum([predictions[i] == 0 for i in range(10000, len(predictions))])
acc_q3 = sum_pred_q3 / (2*len(valid_data))
acc_q3

0.70485

In [32]:
def threshold_and_popularity(threshold_popularity=1/1.5, threshold_jaccard=0.03):
    gameCount = defaultdict(int)
    totalPlayed = 0
    
    for user,game,_ in readJSON("train.json.gz"):
        gameCount[game] += 1
        totalPlayed += 1
        
    mostPopular = [(gameCount[x], x) for x in gameCount]
    mostPopular.sort()
    mostPopular.reverse()
    
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > threshold_popularity: break
    
    correct = 0
    for user, game in valid_data:
        similarities = []
        for d in reviewsPerUser[user]:
            i2 = d[1]
            if i2 == game: continue
            similarities.append(Jaccard(userPerGame[game], userPerGame[i2]))
            
        if max(similarities) > threshold_jaccard and game in return1:
            correct += (game in gamePerUser[user]) # recommend in this case
        else:
            correct += (game not in gamePerUser[user]) # not recommend in this case
        
    return correct/len(valid_data)

In [33]:
acc_q4 = threshold_and_popularity()
acc_q4

0.9945

In [34]:
answers['Q3'] = acc_q3
answers['Q4'] = acc_q4

In [35]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [36]:
threshold_popularity = 1/1.5
threshold_jaccard = 0.003

gameCount = defaultdict(int)
totalPlayed = 0

for user,game,_ in readJSON("train.json.gz"):
    gameCount[game] += 1
    totalPlayed += 1
    
mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPlayed*threshold_popularity: break

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > threshold_popularity: break

predictions = open("HWpredictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    userPlayed = gamePerUser[u]
    similarities = []
    # Logic...
    for d in reviewsPerUser[user]:
        i2 = d[1]
        if i2 == game: continue
        similarities.append(Jaccard(userPerGame[game], userPerGame[i2]))

    if max(similarities) > threshold_jaccard and game in return1:
        predictions.write(u + ',' + g + ',' + "1" + '\n')
    else:
        predictions.write(u + ',' + g + ',' + "0" + '\n')

predictions.close()

In [37]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [38]:
##################################################
# Hours played prediction                        #
##################################################

In [133]:
### Question 6

In [168]:
# I first tried the TF library, but it's not working...

In [39]:
# userIDs = {}
# itemIDs = {}
# interactions = []

# for d in allHours:
#     u = d[0]
#     i = d[1]
#     r = d[2]['hours_transformed']
#     if not u in userIDs: userIDs[u] = len(userIDs)
#     if not i in itemIDs: itemIDs[i] = len(itemIDs)
#     interactions.append((u,i,r))

In [40]:
# random.shuffle(interactions)
# len(interactions)

174999

In [41]:
# nTrain = 165000
# nTest = len(interactions) - nTrain
# interactionsTrain = interactions[:nTrain]
# interactionsTest = interactions[nTrain:]

In [42]:
# itemsPerUser = defaultdict(list)
# usersPerItem = defaultdict(list)
# for u,i,r in interactionsTrain:
#     itemsPerUser[u].append(i)
#     usersPerItem[i].append(u)

In [44]:
# mu = sum([r for _,_,r in interactionsTrain] + [r for _,_,r in interactionsTest])*1.0 / len(allHours)
# mu

3.7178639189242024

In [45]:
# optimizer = tf.keras.optimizers.Adam(0.1)

In [46]:
# class LatentFactorModel(tf.keras.Model):
#     def __init__(self, mu, lamb):
#         super(LatentFactorModel, self).__init__()
#         # Initialize to average
#         self.alpha = tf.Variable(mu)
#         # Initialize to small random values
#         self.betaU = tf.Variable(tf.zeros(len(userIDs)))
#         self.betaI = tf.Variable(tf.zeros(len(itemIDs)))
#         # self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
#         # self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
#         self.lamb = lamb

#     # Prediction for a single instance (useful for evaluation)
#     def predict(self, u, i):
#         p = self.alpha + self.betaU[u] + self.betaI[i] 
#         # + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
#         return p

#     # Regularizer
#     def reg(self):
#         return self.lamb * (tf.reduce_sum(self.betaU**2) +\
#                             tf.reduce_sum(self.betaI**2))
#                         # + tf.reduce_sum(self.gammaU**2) +\
#                         #     tf.reduce_sum(self.gammaI**2))
    
#     # Prediction for a sample of instances
#     def predictSample(self, sampleU, sampleI):
#         u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
#         i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
#         beta_u = tf.nn.embedding_lookup(self.betaU, u)
#         beta_i = tf.nn.embedding_lookup(self.betaI, i)
#         # gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
#         # gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
#         pred = self.alpha + beta_u + beta_i
#             #    tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
#         return pred
    
#     # Loss
#     def call(self, sampleU, sampleI, sampleR):
#         pred = self.predictSample(sampleU, sampleI)
#         r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
#         return tf.nn.l2_loss(pred - r) / len(sampleR)

In [47]:
# modelLFM = LatentFactorModel(mu, 1) # with lambda equal to 1

In [48]:
# modelLFM.trainable_variables[1]

<tf.Variable 'Variable:0' shape=(6710,) dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>

In [49]:
# def trainingStep(model, interactions):
#     Nsamples = 50000
#     with tf.GradientTape() as tape:
#         sampleU, sampleI, sampleR = [], [], []
#         for _ in range(Nsamples):
#             u,i,r = random.choice(interactions)
#             sampleU.append(userIDs[u])
#             sampleI.append(itemIDs[i])
#             sampleR.append(r)

#         loss = model(sampleU,sampleI,sampleR)
#         loss += model.reg()
#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients((grad, var) for
#                               (grad, var) in zip(gradients, model.trainable_variables)
#                               if grad is not None)
#     return loss.numpy()

In [50]:
# for i in range(100):
#     obj = trainingStep(modelLFM, interactionsTrain)
#     if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 26.016861
iteration 20, objective = 4.7072783
iteration 30, objective = 3.285305
iteration 40, objective = 3.3738892
iteration 50, objective = 2.998368
iteration 60, objective = 2.7378821
iteration 70, objective = 2.668524
iteration 80, objective = 2.664396
iteration 90, objective = 2.6570597
iteration 100, objective = 2.6578722


In [436]:
# predictions = []
# for u,g,r in interactionsTest:
#     predict = modelLFM.predict(userIDs[u], itemIDs[g]).numpy()
#     predictions.append(predict)
# len(predictions)

9999

In [437]:
# def MSE(preds, labels):
#     diff = [(x-y)**2 for x,y in zip(preds, labels)]
#     return sum(diff) / len(diff)

In [438]:
# validlabels = [r for _,_,r in interactionsTest]
# validMSE = MSE(predictions,validlabels)
# validMSE

5.225517849604005

In [39]:
trainHours = [r[2]['hours_transformed'] for r in hoursTrain]
globalAverage = sum(trainHours) * 1.0 / len(trainHours)

In [40]:
hoursPerUser, hoursPerItem,Rui = {}, {}, {}
for u,g,d in hoursTrain:
    if u not in hoursPerUser:
        hoursPerUser[u] = [(g, d['hours_transformed'])]
    else:
        hoursPerUser[u].append((g, d['hours_transformed']))
    if g not in hoursPerItem:
        hoursPerItem[g] = [(u, d['hours_transformed'])]
    else:
        hoursPerItem[g].append((u, d['hours_transformed']))
    Rui[(u,g)] = d['hours_transformed']

In [41]:
len(hoursPerItem)

2437

In [42]:
betaU = {}
betaI = {}
for u in hoursPerUser:
    betaU[u] = 0

for g in hoursPerItem:
    betaI[g] = 0

In [43]:
alpha = globalAverage # Could initialize anywhere, this is a guess

In [44]:
def update_alpha():
    global alpha
    num = sum(Rui[(u,g)]- (betaU[u] + betaI[g]) for u,g,_ in hoursTrain)
    denom = len(hoursTrain)
    alpha = num / denom

In [45]:
def update_betaU(lamb):
    global alpha
    for user in hoursPerUser:
        num = sum(Rui[(user,g)]-(alpha+betaI[g]) for g,t in hoursPerUser[user])
        denom = lamb + len(hoursPerUser[user])
        betaU[user] = num / denom

In [46]:
def update_betaI(lamb):
    global alpha
    for item in hoursPerItem:
        num = sum(Rui[(u,item)]-(alpha+betaU[u]) for u,t in hoursPerItem[item])
        denom = lamb + len(hoursPerItem[item])
        betaI[item] = num / denom

In [47]:
def predict(user, item):
    global alpha
    if user not in hoursPerUser and item not in hoursPerItem:
        return alpha
    if user in hoursPerUser and item not in hoursPerItem:
        return alpha + betaU[user]
    if user not in hoursPerUser and item in hoursPerItem:
        return alpha + betaI[item]
    return alpha + betaU[user] + betaI[item]

In [48]:
def MSE():
    mse = sum((_['hours_transformed']-predict(u,i))**2 for u,i,_ in hoursValid) / len(hoursValid)
    return mse

In [49]:
def iterate(lamb, max_iteration=50):
    for iter in range(max_iteration):
        update_alpha()
        update_betaU(lamb)
        update_betaI(lamb)
        if iter % 10 == 9:
            validMSE = MSE()
            print(f"Current Iteration is {iter+1} | MSE: {validMSE}")
    return validMSE

In [50]:
validMSE = iterate(lamb=1)

Current Iteration is 10 | MSE: 3.0071926265486812
Current Iteration is 20 | MSE: 3.0071983181788915
Current Iteration is 30 | MSE: 3.007207678843451
Current Iteration is 40 | MSE: 3.0072192105803386
Current Iteration is 50 | MSE: 3.0072318058212737


In [176]:
answers['Q6'] = validMSE
answers['Q6']

3.0073392524294853

In [177]:
assertFloat(answers['Q6'])

In [ ]:
### Question 7

In [178]:
betaU

{'u70666506': 0.561017721267805,
 'u18612571': -0.3046105728918799,
 'u34283088': -0.8047924105953433,
 'u16220374': 0.16645867858134822,
 'u01499286': 0.06569118494120813,
 'u73063505': 0.3547348100122898,
 'u29223775': 0.27344236257754706,
 'u44401308': 0.39848579614659657,
 'u45027672': -0.30479428143983633,
 'u33908704': 0.5892052422670299,
 'u27998358': -0.4538130787869238,
 'u36214177': -0.6865768732341309,
 'u73747744': 0.4721814453601131,
 'u97936673': -0.4376261616244375,
 'u25365202': 0.4609311720037873,
 'u08631099': -0.049554694608745155,
 'u52584928': -0.008694260214150812,
 'u09520763': 0.39745764532878963,
 'u04893836': -0.20169032583725838,
 'u58381940': -0.19842263396131646,
 'u85007552': -0.5912503993456444,
 'u48369340': 0.06378806804838105,
 'u79530461': -0.8572463811120159,
 'u44157494': 0.5657907967796565,
 'u70118164': -0.9672246198685392,
 'u21352780': -0.10955044271507301,
 'u74354158': -0.2251903184330083,
 'u67890036': -0.8720179043878814,
 'u90421779': -0.28

In [179]:
max_beta_u_id, max_beta_u = max(betaU, key=betaU.get), max(betaU.values())
min_beta_u_id, min_beta_u = min(betaU, key=betaU.get), min(betaU.values())
max_beta_I_id, max_beta_I = max(betaI, key=betaI.get), max(betaI.values())
min_beta_I_id, min_beta_I = min(betaI, key=betaI.get), min(betaI.values())


In [180]:
print("Maximum betaU = " + str(max_beta_u_id) + ' (' + str(max_beta_u) + ')')
print("Maximum betaI = " + str(max_beta_I_id) + ' (' + str(max_beta_I) + ')')
print("Minimum betaU = " + str(min_beta_u_id) + ' (' + str(min_beta_u) + ')')
print("Minimum betaI = " + str(min_beta_I_id) + ' (' + str(min_beta_I) + ')')

Maximum betaU = u60898505 (5.827861878106079)
Maximum betaI = g17604638 (5.456289047184816)
Minimum betaU = u13037838 (-3.005956949143559)
Minimum betaI = g84397720 (-2.848737738548173)


In [181]:
answers['Q7'] = [max_beta_u, min_beta_u, max_beta_I, min_beta_I]
answers['Q7']

[5.827861878106079, -3.005956949143559, 5.456289047184816, -2.848737738548173]

In [ ]:
assertFloatList(answers['Q7'], 4)

In [ ]:
### Question 8

In [193]:
def iterate(lamb, max_iteration=50):
    for iter in range(max_iteration):
        update_alpha()
        update_betaU(lamb)
        update_betaI(lamb)
        if iter == 29:
            validMSE = MSE()
            print(f"Current Lambda is {lamb} | MSE: {validMSE}")
    return validMSE

In [194]:
# Better lambda...
bestValidMSE = None
bestLamb = 0
for lamb in np.arange(0, 10, 0.5):
    validMSE = iterate(lamb, max_iteration=30)
    if bestValidMSE == None or validMSE < bestValidMSE:
        bestValidMSE = validMSE
        bestLamb = lamb

Current Lambda is 0.0 | MSE: 3.020383624199873
Current Lambda is 0.5 | MSE: 3.013181679928504
Current Lambda is 1.0 | MSE: 3.007362495687221
Current Lambda is 1.5 | MSE: 3.0026894312170103
Current Lambda is 2.0 | MSE: 2.9989808264257367
Current Lambda is 2.5 | MSE: 2.9960953818232614
Current Lambda is 3.0 | MSE: 2.993919461048325
Current Lambda is 3.5 | MSE: 2.992358564301318
Current Lambda is 4.0 | MSE: 2.991332895960878
Current Lambda is 4.5 | MSE: 2.9907747814251913
Current Lambda is 5.0 | MSE: 2.9906266263152785
Current Lambda is 5.5 | MSE: 2.9908391701019776
Current Lambda is 6.0 | MSE: 2.991370049865965
Current Lambda is 6.5 | MSE: 2.9921826478452993
Current Lambda is 7.0 | MSE: 2.9932451656309085
Current Lambda is 7.5 | MSE: 2.9945298734105688
Current Lambda is 8.0 | MSE: 2.996012495647068
Current Lambda is 8.5 | MSE: 2.997671704610793
Current Lambda is 9.0 | MSE: 2.999488700097838
Current Lambda is 9.5 | MSE: 3.0014468586000094


In [196]:
answers['Q8'] = (bestLamb, bestValidMSE)
answers['Q8']

(5.0, 2.9906266263152785)

In [197]:
assertFloatList(answers['Q8'], 2)

In [199]:
predictions = open("HWpredictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    if u in betaU and g in betaI:
        predictions.write(u + ',' + g + ',' + str(alpha + betaU[u] + betaI[g]) + '\n')
    else:
        predictions.write(u + ',' + g + ',' + str(0) + '\n')

predictions.close()

In [200]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()